In [1]:
import os, sys, threading
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, Executor
import threading
import re
import csv
from collections import namedtuple
from path import Path
from internetarchive import download, get_item, search_items, get_files

In [2]:
df_usfedcourts = pd.read_csv('/Users/admin/Downloads/ia_search_format.csv', iterator=False, chunksize=1000)

In [ ]:
for chunk in pd.read_csv('/Users/admin/Downloads/ia_search_format.csv', iterator=False, chunksize=1000):
    identifiers = chunk['identifier']
    print(identifiers.head())

In [ ]:
df_usfedcourts = pd.read_csv('/Users/admin/Downloads/ia_search_format.csv')

In [ ]:
df_usfedcourts= df_usfedcourts.sort_values('identifier')

In [ ]:
df_usfedcourts

In [ ]:
df_usfedcourts = df_usfedcourts.set_index(keys='identifier', drop=True)

In [ ]:
df_usfedcourts.columns

In [ ]:
df_downloaded = pd.DataFrame(os.listdir('/Users/admin/Documents/Bulk Legal Data/ia_s3_files'))

In [ ]:
df_downloaded.columns=['identifier']

In [ ]:
df_usfedcourts.append(df_downloaded)

In [ ]:
unique_idents = df_usfedcourts['identifier'].unique

In [ ]:
unique_idents

In [ ]:
ident_list = ([ident for ident in df_usfedcourts['identifier'] if ident not in idents_down])

In [ ]:
def ia_get_filelist(ident_list):
    
    thread_local = threading.local()
    file_list=[]
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:
        item = (executor.map(get_item, ident_list))
        for i in item:
            file_list.append(i.files)
            
    
    return file_list

In [ ]:
with open('dict.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    for i in range(len(cand_files)):
        for name in range(len(cand_files[i])):
            file_dict = cand_files[i][name]
            for key, value in file_dict.items():
                writer.writerow([key, value])

In [ ]:
df_sample = df_usfedcourts.sample(n=1000, random_state=1)
df_sample.set_index(keys='identifier', drop=True)

In [ ]:
def get_ia_filenames(identifiers):
    extensions=('.pdf', '.xml', '.html', '.json')
    return [file['name'] for ident in identifiers for file in get_item(ident).files if file['name'].endswith(extensions)]

def threaded_get_ia_filelist(ident):
    ident = ident
    with ThreadPoolExecutor(max_workers=50) as executor:
        future = executor.submit(get_ia_filenames, ident)
        return future.result()
        #return future

In [ ]:
from pathlib import Path
import urllib.request
from concurrent.futures import ThreadPoolExecutor


def download_one(url):
    """
    Downloads the specified URL and saves it to disk
    """

    req = urllib.request.urlopen(url)
    fullpath = Path(url)
    fname = fullpath.name
    ext = fullpath.suffix

    if not ext:
        raise RuntimeError("URL does not contain an extension")

    with open(fname, "wb") as handle:
        while True:
            chunk = req.read(1024)
            if not chunk:
                break
            handle.write(chunk)

    msg = f"Finished downloading {fname}"
    return msg



def download_all(urls):
    """
    Create a thread pool and download specified urls
    """

    futures_list = []
    results = []

    with ThreadPoolExecutor(max_workers=13) as executor:
        for url in urls:
            futures = executor.submit(download_one, url)
            futures_list.append(futures)

        for future in futures_list:
            try:
                result = future.result(timeout=60)
                results.append(result)
            except Exception:
                results.append(None)
    return results


if __name__ == "__main__":
    urls = (
        "http://www.irs.gov/pub/irs-pdf/f1040.pdf",
        "http://www.irs.gov/pub/irs-pdf/f1040a.pdf",
        "http://www.irs.gov/pub/irs-pdf/f1040ez.pdf",
        "http://www.irs.gov/pub/irs-pdf/f1040es.pdf",
        "http://www.irs.gov/pub/irs-pdf/f1040sb.pdf",
    )

    results = download_all(urls)
    for result in results:
        print(result)



In [ ]:
import concurrent.futures
import threading

def ia_get_filelist(ident_list):
    extensions=('.pdf', '.xml', '.html', '.json')
    thread_local = threading.local()
    file_list=[]
    
    with open('ia_file_list.csv', 'a') as f:
        writer = csv.writer(f, delimiter='\n')
        with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:
            try: 
                item = (executor.map(get_item, ident_list))
                writer.writerow([file['name'] for i in item for file in i.files if file['name'].endswith(extensions)])
            except:
                pass

        
    
    #return [file['name'] for ident in identifiers for file in get_item(ident).files if file['name'].endswith(extensions)]
        #for i in item:
        #    file_list.append(i.files)
            
    return None
    #return file_list
            

In [ ]:
ia_get_filelist(df_sample['identifier'])

In [ ]:
df_file_list = pd.read_csv('/Users/admin/ia_file_list.csv')

In [ ]:
df_file_list.shape

In [ ]:
sample_file_list =  df_file_list.sample(n=1000, random_state=1)

In [ ]:
[print(name) for name in sample_file_list['file_name']]

In [ ]:
for chunk in pd.read_csv('/Users/admin/Downloads/ia_search_format.csv', iterator=False, chunksize=1000):
   # print(chunk['identifier'])
    identifiers = chunk['identifier']
    print(identifiers)
    ia_get_filelist(identifiers)
    print('next chunk')
    

In [ ]:
threaded_get_ia_filelist(df_sample['identifier'])

In [ ]:
import concurrent.futures
import threading

def get_ia_filelist_submit(ident_list):
    extensions=('.pdf', '.xml', '.html', '.json')
    file_list=[]
    
    with open('ia_file_list.csv', 'w') as f:
        writer = csv.writer(f, delimiter='\n')
        with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:
            future = executor.submit(get_item, ident_list)
            print(future.result())
                

            
            #writer.writerow([file['name'] for i in item for file in i.files if file['name'].endswith(extensions)])
    

In [ ]:
ia_get_filelist(df_usfedcourts['identifier'])

In [ ]:
get_ia_filelist_submit(df_sample['identifier'])

In [ ]:
df_usfedcourts

In [ ]:
test_item= get_item('gov.uscourts.miwd.84825')

In [ ]:
test_item.files

In [ ]:
[file['name'] for ident in identifiers for file in sample_get_filelist]

In [ ]:
sample_file_list = threaded_get_ia_filelist(df_sample['identifier'])

In [ ]:
sample_file_list.result()

In [ ]:
ident = re.findall(r'gov\.uscourts\.[a-z]+\.[0-9]+', 'gov.uscourts.ilnb.1095107.1.0.pdf')

In [ ]:
idents = [re.findall(r'gov\.uscourts\.[a-z]+\.[0-9]+', ident) for ident in sample_file_list.result()]

In [ ]:
idents

In [ ]:
result = threaded_get_ia_files_test(df_sample['identifier'])

In [3]:
def create_paths(file):
    Paths = namedtuple('Paths', ['court', 'ident', 'new_dir', 'file_path', 'file_url'])
    base_url = 'http://s3.us.archive.org'
    base_path = '/Users/admin/Documents/Bulk Legal Data/ia_s3_files2'
    #regex to parse a case identifier and extract just the court name
    court = re.search(r'gov\.uscourts\.[a-z]+', file).group()
    ident = re.search(r'gov\.uscourts\.[a-z]+\.[0-9]+', file).group()
    
    new_dir = Path.joinpath(base_path, court)
    file_path = Path.joinpath(base_path, court, file)
    file_url = Path.joinpath(base_url, ident, file)
    
    return Paths(court=court, ident=ident, new_dir=new_dir, file_path=file_path, file_url=file_url)
       
def create_dir(new_dir):
    if not os.path.isdir(new_dir):
        new_dir.mkdir()
    else:
        #print("directory exists \n")
        pass
    
    return None
    
def download_ia_file( file_path, file_url):
    s = requests.session()
    
    if not os.path.isfile(file_path):
        #print('downloading:', file, '\n')
        try:
            r = s.get(file_url)
            print(file_url, r.status_code)
            with open(file_path, 'wb') as f:
                f.write(r.content)
        except:
            print('error downloading:', file, '\n')
            #pass
    else:
        #print('file already exists')
        pass
    return None

def get_files(file):
    Paths = create_paths(file=file)
    court = Paths.court
    new_dir = Paths.new_dir
    ident = Paths.ident
    file_path = Paths.file_path
    file_url = Paths.file_url
    #print(new_dir, file_path)
    #print(court, new_dir, ident, file_path, file_url )
    create_dir(new_dir=new_dir)
    download_ia_file(file_path=file_path, file_url=file_url)
    return None

In [4]:
import concurrent.futures
import threading

def threaded_get_files(file_list):
    with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:
        return executor.map(get_files, file_list)

In [ ]:
threaded_get_files(sample_file_list['file_name'])

In [5]:
for count, chunk in enumerate(pd.read_csv('/Users/admin/ia_file_list.csv', iterator=False, chunksize=10000)):
    print('chunk: ', count)
    #print(chunk['identifier'])
    file_names = chunk['file_name']
    #[print(file) for file in file_names]
    threaded_get_files(file_names)
    
    

chunk:  0
chunk:  1
chunk:  2
chunk:  3
chunk:  4
chunk:  5
chunk:  6
chunk:  7
chunk:  8
chunk:  9
chunk:  10
chunk:  11
chunk:  12
chunk:  13
chunk:  14
chunk:  15
chunk:  16
chunk:  17
chunk:  18
chunk:  19
chunk:  20
chunk:  21
chunk:  22
chunk:  23
chunk:  24
chunk:  25
chunk:  26
chunk:  27
chunk:  28
chunk:  29
chunk:  30
chunk:  31
chunk:  32
chunk:  33
chunk:  34
chunk:  35
chunk:  36
chunk:  37
chunk:  38
chunk:  39
chunk:  40
chunk:  41
chunk:  42
chunk:  43
chunk:  44
chunk:  45
chunk:  46
chunk:  47
chunk:  48
chunk:  49
chunk:  50
chunk:  51
chunk:  52
chunk:  53
chunk:  54
chunk:  55
chunk:  56
chunk:  57
chunk:  58
chunk:  59
chunk:  60
chunk:  61
http://s3.us.archive.org/gov.uscourts.ilnb.1258147/gov.uscourts.ilnb.1258147_meta.xml 200
http://s3.us.archive.org/gov.uscourts.ilnb.1275576/gov.uscourts.ilnb.1275576.1.0.pdf 200
chunk:  62
http://s3.us.archive.org/gov.uscourts.ilnb.1350521/gov.uscourts.ilnb.1350521_files.xml 200
chunk:  63
chunk:  64
chunk:  65
chunk:  66
chu

KeyboardInterrupt: 

In [ ]:
get_files('gov.uscourts.casd.473935_files.xml')
#paths = create_paths('gov.uscourts.casd.473935_files.xml')

In [ ]:
[get_files(i) for i in sample_file_list.result()]

In [ ]:
create_dir('/Users/admin/Documents/test')

In [ ]:
from collections import namedtuple
def create_paths_tuple(ident):
    Paths = namedtuple('Paths', ['court', 'ident', 'new_dir', 'file_path', 'file_url'])
    base_url = 'https://s3.us.archive.org'
    base_path = '/Users/admin/Documents/Bulk Legal Data/ia_s3_files'
    #regex to parse a case identifier and extract just the court name
    court = re.search(r'gov\.uscourts\.[a-z]+', ident).group()
    
    new_dir = Path.joinpath(base_path, court)
    file_path = Path.joinpath(base_path, court, ident)
    file_url = Path.joinpath(base_url, ident)
    
    return Paths(court=court, ident=ident, new_dir=new_dir, file_path=file_path, file_url=file_url)

In [ ]:
create_paths_tuple('gov.uscourts.casd.473935_files.xml')

In [ ]:
Paths = create_paths_tuple('gov.uscourts.casd.473935_files.xml')

In [ ]:
def download_ia_file(file_path, file_url):
    s = requests.session()
    
    if not os.path.isfile(file_path):
        r = s.get(file_url)
        with open(file_path, 'wb') as f:
            f.write(r.content)
    else:
        print('file already exists')

In [ ]:
 try:
        n
        print(new_dir)

        for item in items.files: 
            if item['name'].endswith(extensions):
                #print(item['name'])
                file_url = Path.joinpath(base_url, identifier, item['name'])
                print(file_url)
                file_name = Path.joinpath(file_path, identifier, item['name'])
                print(file_name)

    except:
        print(identifier, 'was already downloaded')

In [ ]:
create_path('gov.uscourts.casd.47393')

In [ ]:
cour2t = re.search(r'gov\.uscourts\.[a-z]+', 'gov.uscourts.casd.47393').group()

In [ ]:
cour2t

In [ ]:
def get_paths(ident):
    #print(ident)
    x = ident.rsplit('.')
    
    identifier = x[0] + '.' + x[1] + '.' +  x[2] + '.' +  x[3]
    #print(identifier)
    
    base_url = 'https://s3.us.archive.org'
    file_path = '/Users/admin/Documents/Bulk Legal Data/ia_s3_files'
    new_dir = Path.joinpath(file_path, identifier)
    file_url = Path.joinpath(base_url, identifier)
    #print(file_url)
    
    file_name = Path.joinpath(file_path, identifier)
    #print(file_name)
    #if os.path.isfile(file_name):
    #    print(filename, 'was already downloaded')  
    #else:
    #    r = s.get(file_url)
    #    with open(file_name, 'wb') as f:
    #        f.write(r.content)
    return identifier, file_url, file_name

In [ ]:
[get_paths(ident) for ident in sample_list]
#[get_ia_s3_file(filename) for file in sample_list]
#[get_ia_s3_file(filename) for result in threaded_get_ia_files_test(df_sample['identifier'].tolist())]

In [ ]:
fieldnames =['identifier','filename']
with open('ia_filenames.csv', 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=field_names)
    writer.writeheader()
    [writer.writerow(item['identifier']) for ident in df_sample['identifier']]
    [writer.writerow(file['name']) for ident in df_sample['identifier'] for file in get_item(ident).files if file['name'].endswith(extensions)]

In [ ]:
def threaded_get_ia_files(identifiers):
    with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:
        item = executor.map(get_item, identifiers)
        print(identifiers)
        

In [ ]:
print(threaded_get_ia_files(identifiers=df_sample['identifier']))

In [ ]:
sample_list = df_sample['identifier'].tolist()

In [ ]:
type(sample_list)

In [ ]:
def get_ia_items(identifiers):
    return [get_item(idents).files for idents in identifiers]

In [ ]:
def get_ia_filenames(identifier):
    extensions=('.pdf', '.xml', '.html', '.json')
    return [file['name'] for file in get_item(identifier).files if file['name'].endswith(extensions)]

In [ ]:
get_ia_filenames('gov.uscourts.wvsd.65368')

In [ ]:
def get_ia_filenames_threaded(identifier):
    with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:
        print(identifier)
        return executor.map(get_ia_filenames(identifier))
    

In [ ]:
executor?

In [ ]:
get_ia_filenames_threaded(sample_list)

In [ ]:
filename_list = [get_ia_filenames(ident) for ident in sample_list]

In [ ]:
import threading
import urllib.request
import time

def downloadImage(imagePath, fileName):
    print("Downloading Image from ", imagePath)
    urllib.request.urlretrieve(imagePath, fileName)
    print("Completed Download")

def executeThread(i): 
    imageName = "temp/image-" + str(i) + ".jpg"
    downloadImage("http://lorempixel.com/400/200/sports", imageName)

def main():
    t0 = time.time()
    # create an array which will store a reference to
    # all of our threads
    threads = []
    # create 10 threads, append them to our array of threads
    # and start them off
    for i in range(10):
    thread = threading.Thread(target=executeThread, args=(i,))
    threads.append(thread)
    thread.start()

    # ensure that all the threads in our array have completed
    # their execution before we log the total time to complete
    for i in threads:
    i.join()
    # calculate the total execution time
    t1 = time.time()
    totalTime = t1 - t0
    print("Total Execution Time {}".format(totalTime))

    if __name__ == '__main__':
    main()

In [ ]:
[print(type(i)) for i['name'] in [get_ia_item(ident) for ident in df_sample['identifier']]]

In [ ]:
print(files)

In [ ]:
def threaded_get_ia_item(ident):
    with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:
        executor.map(get_ia_item, ident)

In [ ]:
threaded_get_ia_item(df_sample['identifier'])

In [ ]:
[download(identifier=ident, formats=('Text PDF','Image Container PDF','HTML','JSON')) for ident in df_sample['identifier']]

In [ ]:
import concurrent.futures
import threading

def ia_get_files(ident_list, directory):
    os.chdir(directory)
    with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:
        executor.map(download, ident_list)

In [ ]:
ia_get_files(ident_list=df_usfedcourts['identifier'], directory = '/Users/admin/Documents/Bulk Legal Data/ia_s3_files')

In [ ]:
def ia_get_files2(ident_list, directory):
    os.chdir(directory)
    items = [get_item(identifier).files for identifier in ident_list]
    with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:

In [ ]:


items = get_item(identifier)

                
                for item in items.files: 
                    if item['name'].endswith(extensions):
                        #print(item['name'])
                        file_url = Path.joinpath(base_url, identifier, item['name'])
                        print(file_url)
                        file_name = Path.joinpath(file_path, identifier, item['name'])
                        print(file_name)
                        download_item(file_url=file_url, file_name=file_name)
                        
            except:
                print(identifier, 'was already downloaded')

            #else:
                
        else:
            print(identifier, 'has no pdf files')
        

In [ ]:
def download_item(file_url, file_name):
    if os.path.isfile(file_name):
        print(item['name'], 'was already downloaded')  
    else:
        s = requests.session()
        r = s.get(file_url)
        with open(file_name, 'wb') as f:
            f.write(r.content) 

In [ ]:
download?

In [ ]:
def download_item(file_url, file_name):
    if os.path.isfile(file_name):
        print(item['name'], 'was already downloaded')  
    else:
        s = requests.session()
        r = s.get(file_url)
        with open(file_name, 'wb') as f:
            f.write(r.content)    

In [ ]:
def get_ia_s3_files(ia_idents):
    base_url = 'https://s3.us.archive.org'
    file_path = '/Users/admin/Documents/Bulk Legal Data/ia_s3_files'
    extensions = ('.pdf', '.xml', '.html')
    s = requests.session()
    total_results = len(ia_idents)
    processed_results = 0
    new_dir = Path.joinpath(file_path, identifier)
    
    try:
        new_dir.mkdir()
        print(new_dir)

        for item in items.files: 
            if item['name'].endswith(extensions):
                #print(item['name'])
                file_url = Path.joinpath(base_url, identifier, item['name'])
                print(file_url)
                file_name = Path.joinpath(file_path, identifier, item['name'])
                print(file_name)
                if os.path.isfile(file_name):
                    print(item['name'], 'was already downloaded')  
                else:
                      #urllib.request.urlretrieve(file_url, filename=file_name)
                    #r = requests.get(file_url)
                    r = s.get(file_url)
                    with open(file_name, 'wb') as f:
                        f.write(r.content)
    except:
        print(identifier, 'was already downloaded')



In [ ]:
def get_ia_s3(ia_search):
    base_url = 'https://s3.us.archive.org'
    file_path = '/Users/admin/Documents/Bulk Legal Data/ia_s3_3.1'
    extensions = ('.pdf', '.xml', '.html')
    s = requests.session()
    total_results = len(ia_search)
    processed_results = 0
    for result in ia_search:
        processed_results +=1
        print(total_results - processed_results, 'identifiers remaining')
        identifier = result['identifier']
        items = get_item(identifier)
        has_pdf = [item for item in items.files if item['name'].endswith('pdf')]
        if has_pdf:
            print(identifier, 'has', len(has_pdf), 'pdf file(s)')
            new_dir = Path.joinpath(file_path, identifier)
            try:
                new_dir.mkdir()
                print(new_dir)
                
                for item in items.files: 
                    if item['name'].endswith(extensions):
                        #print(item['name'])
                        file_url = Path.joinpath(base_url, identifier, item['name'])
                        print(file_url)
                        file_name = Path.joinpath(file_path, identifier, item['name'])
                        print(file_name)
                        if os.path.isfile(file_name):
                            print(item['name'], 'was already downloaded')  
                        else:
                              #urllib.request.urlretrieve(file_url, filename=file_name)
                            #r = requests.get(file_url)
                            r = s.get(file_url)
                            with open(file_name, 'wb') as f:
                                f.write(r.content)
            except:
                print(identifier, 'was already downloaded')

            #else:
                
        else:
            print(identifier, 'has no pdf files')
        


In [ ]:
def get_ia_s3_1(ia_search):
    base_url = 'https://s3.us.archive.org'
    file_path = '/Users/admin/Documents/Bulk Legal Data/ia_s3_3.1'
    extensions = ('.pdf', '.xml', '.html')
    total_results = len(ia_search)
    processed_results = 0
    for result in ia_search:
        processed_results +=1
        print(total_results - processed_results, 'identifiers remaining')
        identifier = result['identifier']
        items = get_item(identifier)
        has_pdf = [item for item in items.files if item['name'].endswith('pdf')]
        if has_pdf:
            print(identifier, 'has', len(has_pdf), 'pdf file(s)')
            new_dir = Path.joinpath(file_path, identifier)
            try:
                new_dir.mkdir()
                print(new_dir)
                
                for item in items.files: 
                    if item['name'].endswith(extensions):
                        #print(item['name'])
                        file_url = Path.joinpath(base_url, identifier, item['name'])
                        print(file_url)
                        file_name = Path.joinpath(file_path, identifier, item['name'])
                        print(file_name)
                        download_item(file_url=file_url, file_name=file_name)
                        
            except:
                print(identifier, 'was already downloaded')

            #else:
                
        else:
            print(identifier, 'has no pdf files')
        


In [ ]:
search_flmb = search_items('title:(gov.uscourts.flmb) AND collection:(usfederalcourts)')

In [ ]:
get_ia_s3_1(search_flmb)

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from requests_futures.sessions import FuturesSession

session = FuturesSession()

In [ ]:
dict = {}
[get_item(result['identifier']).item_metadata['files'])